In [1]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import backend as K

In [2]:
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    train_images = mnist_train_images.reshape(mnist_train_images.shape[0], 1, 28, 28)
    test_images = mnist_test_images.reshape(mnist_test_images.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    train_images = mnist_train_images.reshape(mnist_train_images.shape[0], 28, 28, 1)
    test_images = mnist_test_images.reshape(mnist_test_images.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)

In [3]:
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255

train_labels = tensorflow.keras.utils.to_categorical(mnist_train_labels, 10)
test_labels = tensorflow.keras.utils.to_categorical(mnist_test_labels, 10)

In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# 64 3x3 kernels
model.add(Conv2D(64, (3, 3), activation='relu'))

# Reduce by taking the max of each 2x2 block
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout to avoid overfitting
model.add(Dropout(0.25))

# Flatten the results to one dimension for passing into our final layer
model.add(Flatten())

# A hidden layer to learn with
model.add(Dense(128, activation='relu'))

# Another dropout
model.add(Dropout(0.5))

# Final categorization from 0-9 with softmax
model.add(Dense(10, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=5, verbose=2, 
                    validation_data=(test_images, test_labels))

score = model.evaluate(test_images, test_labels, verbose=0)

Epoch 1/5
1875/1875 - 124s - loss: 0.1874 - accuracy: 0.9428 - val_loss: 0.0448 - val_accuracy: 0.9853
Epoch 2/5
1875/1875 - 134s - loss: 0.0815 - accuracy: 0.9764 - val_loss: 0.0356 - val_accuracy: 0.9875
Epoch 3/5
1875/1875 - 138s - loss: 0.0617 - accuracy: 0.9814 - val_loss: 0.0307 - val_accuracy: 0.9900
Epoch 4/5
1875/1875 - 128s - loss: 0.0488 - accuracy: 0.9847 - val_loss: 0.0291 - val_accuracy: 0.9910
Epoch 5/5
1875/1875 - 127s - loss: 0.0415 - accuracy: 0.9865 - val_loss: 0.0251 - val_accuracy: 0.9909


In [7]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.025144757702946663
Test accuracy: 0.9908999800682068
